 #  Muhammad Haziq Ijaz i212692


### Reading File

In [22]:
import csv

def load_text(file_path):
    with open(file_path, 'rt', encoding="utf-8") as f:
        reader = csv.reader(f)
        passage = list(reader)
    return passage[0][0]

# Load text from file
text = load_text('urdu-corpus.txt')

print("Original text (first 500 characters):")
print(text[:500])

Original text (first 500 characters):
گزشتہ کئی سالوں سے مختلف بحران آتے جاتے رہے لیکن حالیہ آٹا ، چینی سمیت دیگر بحران اچانک پید ا ہوئے اور ان پر جے آئی ٹی تشکیل دے دیں گئیں تاکہ عوام کو ریلیف دیا جاسکے دوسری جانب بجلی ، گیس ، پانی سمیت دیگر بلوں میں کئی سو گنا اضافہ کردیا گیا ،صوبائی و وفاقی وزراء نے اپنے اپنے ایوانوں بحران کے ذمہ دار عناصر کو بے نقاب کرنے کے بجائے سب اچھاہے کی رپورٹ پیش کیں ساتھ ہی اورنگزیب کی طرح جواب دیا، حقائق کی ’’قبر ذرا گہری کھودنا!‘‘ تاریخی گواہ ہے بقول ماہرین ، مبصرین اور صحافیوں کا کہنا اور لکھنا ہے کہ پ


### Segmentation Code

**`This part of code detects common patterns that signify the end of a sentence.It splits the text into coherent sentences by using common end words and punctuation.`**








In [23]:
import re
from collections import Counter

def detect_patterns(text):
    common_end_words = [
        "ہے", "تھا", "تھی", "ہیں", "تھے", "کرنا", "کیا", "جاتا", "جاتی", "ہوں",
        "رہا", "رہی", "رہے", "آیا", "آئی", "گیا", "گئی", "دیا", "دیتی", "ہوں گی",
        "ہوں گا", "ہوگی", "ہوگا", "چاہتا", "چاہتی", "چاہیے", "لگتا", "لگتی", "سکتا", "سکتی",
        "گا", "گی", "گے", "والا", "والی", "والے", "کر", "ہو", "تو", "کہ"
    ]
    punctuation = set("۔؟!،:")

    words = re.findall(r'\b\w+\b', text)
    word_freq = Counter(words)
    additional_end_words = [word for word, count in word_freq.most_common(30) if word not in common_end_words]
    common_end_words.extend(additional_end_words)

    return common_end_words, punctuation

def segment_urdu_sentences(text, common_end_words, punctuation):
    patterns = (
        r'([۔؟!])'  # Sentence-ending punctuation
        r'|'  # OR
        r'(\b(?:' + '|'.join(re.escape(word) for word in common_end_words) + r')\b)(?=\s*[' + ''.join(re.escape(p) for p in punctuation) + r']|\s*$)'  # Common end words followed by punctuation or end of text
    )

    segments = re.split(patterns, text)
    segments = [seg.strip() for seg in segments if seg and seg.strip()]

    sentences = []
    current_sentence = ""
    for segment in segments:
        current_sentence += segment + ' '
        if any(p in segment for p in punctuation) or any(segment.endswith(word) for word in common_end_words):
            sentences.append(current_sentence.strip())
            current_sentence = ""

    if current_sentence.strip():
        sentences.append(current_sentence.strip())

    return sentences

# Detect patterns and segment sentences
common_end_words, punctuation = detect_patterns(text)
segmented_sentences = segment_urdu_sentences(text, common_end_words, punctuation)

print(f"\nSegmented sentences (first 5 out of {len(segmented_sentences)}):")
for sentence in segmented_sentences[:5]:
    print(sentence)


Segmented sentences (first 5 out of 36):
گزشتہ کئی سالوں سے مختلف بحران آتے جاتے رہے لیکن حالیہ آٹا ، چینی سمیت دیگر بحران اچانک پید ا ہوئے اور ان پر جے آئی ٹی تشکیل دے دیں گئیں تاکہ عوام کو ریلیف دیا جاسکے دوسری جانب بجلی ، گیس ، پانی سمیت دیگر بلوں میں کئی سو گنا اضافہ کردیا
گیا
،صوبائی و وفاقی وزراء نے اپنے اپنے ایوانوں بحران کے ذمہ دار عناصر کو بے نقاب کرنے کے بجائے سب اچھاہے کی رپورٹ پیش کیں ساتھ ہی اورنگزیب کی طرح جواب
دیا
، حقائق کی ’’قبر ذرا گہری کھودنا


### SentencePiece Code


This function evaluates the segmentation accuracy by comparing the segmented text to the original text.
 It calculates similarity based on character overlap, word preservation, and sentence count ratio.
 text



In [24]:
class SimpleSentencePiece:
    def __init__(self, vocab_size=1000):
        self.vocab_size = vocab_size
        self.vocab = {}
        self.reverse_vocab = {}

    def train(self, sentences):
        all_text = ' '.join(sentences)
        subwords = []
        for i in range(1, 5):  # Consider subwords of length 1 to 4
            subwords.extend([all_text[j:j+i] for j in range(len(all_text)-i+1)])

        subword_freq = Counter(subwords)
        self.vocab = {subword: i for i, (subword, _) in enumerate(subword_freq.most_common(self.vocab_size-1))}
        self.vocab['<UNK>'] = len(self.vocab)
        self.reverse_vocab = {i: subword for subword, i in self.vocab.items()}

    def encode(self, text):
        encoded = []
        i = 0
        while i < len(text):
            for j in range(min(4, len(text) - i), 0, -1):
                if text[i:i+j] in self.vocab:
                    encoded.append(self.vocab[text[i:i+j]])
                    i += j
                    break
            else:
                encoded.append(self.vocab['<UNK>'])
                i += 1
        return encoded

    def decode(self, ids):
        return ''.join(self.reverse_vocab.get(id, '<UNK>') for id in ids)

# Train Simple SentencePiece model
sp_model = SimpleSentencePiece(vocab_size=1000)
sp_model.train(segmented_sentences)

### Encode and Decode

In [25]:
import random

# Select a random sample from segmented sentences for encoding and decoding
sample_text = random.choice([s for s in segmented_sentences if len(s) > 20])

# Encode and decode
encoded = sp_model.encode(sample_text)
decoded = sp_model.decode(encoded)

print("\nSample text:")
print(sample_text)
print("\nEncoded text (first 150 tokens):")
print(encoded[:150])
print("\nDecoded text:")
print(decoded)


Sample text:
، جس کا ذکر آگے چل کر آئے

Encoded text (first 150 tokens):
[58, 739, 119, 272, 145, 980, 9, 638, 157, 928, 8]

Decoded text:
، جس کا ذکر آگے چل کر آئے


### Accuracy

In [26]:
def evaluate_segmentation(original_text, segmented_sentences):
    segmented_text = ' '.join(segmented_sentences)
    similarity = sum(a == b for a, b in zip(original_text, segmented_text)) / max(len(original_text), len(segmented_text))

    original_words = set(re.findall(r'\b\w+\b', original_text))
    segmented_words = set(re.findall(r'\b\w+\b', segmented_text))
    word_preservation = len(original_words.intersection(segmented_words)) / len(original_words)

    original_sentences = len(re.findall(r'[۔؟!]', original_text))
    segmented_sentences_count = len(segmented_sentences)
    sentence_count_ratio = min(segmented_sentences_count / max(original_sentences, 1), 1)

    return {
        'character_similarity': similarity,
        'word_preservation': word_preservation,
        'sentence_count_ratio': sentence_count_ratio,
        'overall_score': (similarity + word_preservation + sentence_count_ratio) / 3
    }

# Evaluate segmentation accuracy
evaluation_results = evaluate_segmentation(text, segmented_sentences)

print("\nSegmentation Evaluation:")
for metric, score in evaluation_results.items():
    print(f"{metric}: {score:.2%}")

tokenization_eval = evaluate_segmentation(sample_text, [decoded])

print("\nTokenization Evaluation:")
for metric, score in tokenization_eval.items():
    print(f"{metric}: {score:.2%}")


Segmentation Evaluation:
character_similarity: 18.20%
word_preservation: 100.00%
sentence_count_ratio: 100.00%
overall_score: 72.73%

Tokenization Evaluation:
character_similarity: 100.00%
word_preservation: 100.00%
sentence_count_ratio: 100.00%
overall_score: 100.00%


In [27]:

    text = (
        "بے چاری عوام چونکہ ہمیشہ سے دھوکہ کھانے کی عادی رہی ہے اس لئے ‘‘تبدیلی سرکار’’ کی چکنی چپڑی باتوں میں آگئی اور اپنے بہتر مستقبل "
        "کے لئے نئی حکومت کو اقتدار کے ایوانوں تک پہنچا دیا"
    )


    common_end_words, punctuation = detect_patterns(text)


    segmented_sentences = segment_urdu_sentences(text, common_end_words, punctuation)


    print("\nSegmented Output:")
    for sentence in segmented_sentences:
        print(sentence)



Segmented Output:
بے چاری عوام چونکہ ہمیشہ سے دھوکہ کھانے کی عادی رہی ہے اس لئے ‘‘تبدیلی سرکار’’ کی چکنی چپڑی باتوں میں آگئی اور اپنے بہتر مستقبل کے لئے نئی حکومت کو اقتدار کے ایوانوں تک پہنچا دیا
